In [ ]:
def unwrap(s):
    # Extract the numerical values into an array 
    # The variable "s" can be of any type
    # then ignore non-numerical els
    v = np.array([])
    if isinstance(s, (int, float, complex)):
        v = np.array([s])
    elif isinstance(s, np.ndarray) and np.issubdtype(s.dtype, np.number):
        v = s.flatten()
    elif isinstance(s, dict):
        v = unwrap(list(s.values()))
    elif isinstance(s, (list, tuple)):
        for item in s:
            v = np.concatenate((v, unwrap(item)))
    return v


def rewrap(s, v):
    # Map the numerical elements from the array onto the variables
    # -can be of any type. 
    # copy non-numericals
    if isinstance(s, (int, float, complex)):
        return v[0], v[1:]
    elif isinstance(s, np.ndarray) and np.issubdtype(s.dtype, np.number):
        return np.reshape(v[:s.size], s.shape), v[s.size:]
    elif isinstance(s, dict):
        values = list(s.values())
        new_values = []
        for value in values:
            new_value, v = rewrap(value, v)
            new_values.append(new_value)
        return dict(zip(s.keys(), new_values)), v
    elif isinstance(s, (list, tuple)):
        new_list = []
        for item in s:
            new_item, v = rewrap(item, v)
            new_list.append(new_item)
        return new_list, v
    else:
        return s, v
        
#phases = (phases + np.pi) % (2 * np.pi) - np.pi

import matplotlib.pyplot as plt
from matplotlib.patches import Patch

def boundedline(*args, alpha=False, ax=None, transparency=0.2, orientation='vert', cmap=None):
    if alpha:
        patch_alpha = transparency
        line_alpha = 1.0
    else:
        patch_alpha = 1.0
        line_alpha = transparency

    if ax is None:
        ax = plt.gca()

    if cmap is not None:
        cmap = plt.get_cmap(cmap)

    lines = []
    patches = []

    n_args = len(args)
    

    for i in range(0, n_args, 4):
        x = args[i]
        y = args[i + 1]
        b = args[i + 2]
        linespec = args[i + 3]

        if isinstance(x, (list, np.ndarray)):
            x = np.asarray(x)
        if isinstance(y, (list, np.ndarray)):
            y = np.asarray(y)
        if isinstance(b, (list, np.ndarray)):
            b = np.asarray(b)

        if len(x.shape) == 1:
            x = x.reshape((-1, 1))
        if len(y.shape) == 1:
            y = y.reshape((-1, 1))

        if orientation == 'horiz':
            x, y = y, x

        if x.shape != y.shape:
            raise ValueError("Invalid x, y shape")

        if len(x) != b.shape[0] or b.ndim > 3:
            raise ValueError("Invalid b shape")

        n_lines = x.shape[1]
        if b.ndim == 2:
            b = np.repeat(b[:, :, np.newaxis], n_lines, axis=2)

        for j in range(n_lines):
            line_x = x[:, j]
            line_y = y[:, j]
            line_b = b[:, :, j]

            lower_bounds = line_y - line_b[:, 0]
            upper_bounds = line_y + line_b[:, 1]

            line = ax.plot(line_x, line_y, linespec, color=cmap(j) if cmap else None, alpha=line_alpha)
            patch = ax.fill_between(line_x, lower_bounds, upper_bounds, alpha=patch_alpha, facecolor=line[0].get_color())

            lines.append(line[0])
            patches.append(patch)

    return lines, patches


import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.collections as mcollections

def outlinebounds(hl, hp, ax):
    hnew = []
    for il in range(len(hp)):
        col = hl[il].get_color()
        if isinstance(hp[il], mlines.Line2D):  # Check if it's a Line2D object
            xy = [hp[il].get_xdata(), hp[il].get_ydata()]
            hnew.append(ax.plot(xy[0], xy[1], linestyle='-', color=col)[0])
        elif isinstance(hp[il], mcollections.PolyCollection):  # Check if it's a PolyCollection object
            xy = hp[il].get_paths()[0].vertices.T
            hnew.append(ax.plot(xy[0], xy[1], linestyle='-', color=col)[0])
    
    return hnew


